In [0]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [0]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [0]:
la = LabelEncoder()
la2 = LabelEncoder()
std = StandardScaler()

In [5]:
total = df.append(test)
total.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [0]:
total['enc_ucd'] = la.fit_transform(total['use_case_for_drug'])

In [0]:
total['enc_nod'] = la2.fit_transform(total['name_of_drug'])

In [0]:
train = total[~total['base_score'].isna()].reset_index(drop=True)
test = total[total['base_score'].isna()].reset_index(drop=True)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train[['enc_nod', 'enc_ucd', 
                                                           'effectiveness_rating',
                                                           'number_of_times_prescribed']], train['base_score'], test_size=0.33, random_state=42)

In [0]:
xar = std.fit_transform(np.array(train[['enc_nod', 'enc_ucd', 'effectiveness_rating', 'number_of_times_prescribed']]))
y = np.array(train['base_score'])

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
x =  Variable(torch.from_numpy(xar))
y =  Variable(torch.tensor(y, dtype=torch.float))

In [0]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(xar, y, test_size=0.33, random_state=42)


input_size = 4
h1 = 128
h2 = 256
h3 = 256
h4 = 256
learning_rate = 0.000001
batch_size = 256
num_epochs = 300

In [0]:
class PrepareData(Dataset):

    def __init__(self, X, y, scale_X=True):
        if not torch.is_tensor(X):
            if scale_X:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
        if not torch.is_tensor(y):
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = PrepareData(np.array(X_train), y=np.array(y_train), scale_X=False)
test_ds = PrepareData(np.array(X_test), y=np.array(y_test), scale_X=False)

train_set = DataLoader(train_ds, batch_size=batch_size,
                       shuffle=True)
test_set = DataLoader(test_ds, batch_size=batch_size,
                      shuffle=True)

class RegressionModel(nn.Module):

    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4):
        super(RegressionModel, self).__init__()
        self.dense_h1 = nn.Linear(in_features=input_size, out_features=hidden_size1)
        self.relu_h1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.3)
        self.dense_h2 = nn.Linear(in_features=hidden_size1, out_features=hidden_size2)
        self.relu_h2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.3)
        self.dense_h3 = nn.Linear(in_features=hidden_size2, out_features=hidden_size3)
        self.relu_h3 = nn.ReLU()
        self.dropout3 = nn.Dropout(p=0.2)
        self.dense_h4 = nn.Linear(in_features=hidden_size3, out_features=hidden_size4)
        self.relu_h4 = nn.ReLU()
        self.dropout4 = nn.Dropout(p=0.2)
        self.dense_out = nn.Linear(in_features=hidden_size4, out_features=1)

    def forward(self, X):

        out = self.relu_h1(self.dense_h1(X))
        out = self.dropout1(out)
        out = self.relu_h2(self.dense_h2(out))
        out = self.dropout2(out)
        out = self.relu_h3(self.dense_h3(out))
        out = self.dropout3(out)
        out = self.relu_h4(self.dense_h4(out))
        out = self.dropout4(out)
        out = self.dense_out(out)
        return out

m = RegressionModel(input_size=input_size, hidden_size1=h1, hidden_size2=h2,
                    hidden_size3=h3, hidden_size4=h4)

cost_func = nn.MSELoss()
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)


In [0]:
for e in range(num_epochs):
    train_loss = 0
    m.train()
    for i, b in enumerate(train_set):

        _X = Variable(b[0]).float()
        _y = Variable(b[1]).float()
        # print(_y)

        #==========Forward pass===============

        preds = m(_X)
        loss = cost_func(preds, _y)

        #==========backward pass==============

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()


    m.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, b in enumerate(test_set):

            _X = Variable(b[0]).float()
            _y = Variable(b[1]).float()

            #apply model
            test_preds = m(_X)
            test_loss = cost_func(test_preds, _y)

            valid_loss += loss.item()
    
    #########################
    ## PRINT EPOCH RESULTS ##
    #########################
    train_loss /= len(train_set)
    valid_loss /= len(test_set)
    print(f'Epoch: {e+1}/{num_epochs}.. Training loss: {train_loss}.. Validation Loss: {valid_loss}')
    # print(f'Training accuracy: {100 * score_func(y_train, train_pred)}, Validation accuracy: {100 * score_func(y_test, test_pred)}')
    

In [0]:
tt = np.array(test[['enc_nod', 'enc_ucd', 'effectiveness_rating', 'number_of_times_prescribed']])

In [53]:
m.eval()
xt =  Variable(torch.from_numpy(std.transform(tt))).float()
m(xt)

tensor([[5.4460],
        [5.7320],
        [5.7179],
        ...,
        [5.7366],
        [5.6295],
        [5.6030]], grad_fn=<AddmmBackward>)

In [27]:
xt[0]

tensor([1406.,  193.,   10.,   22.])